In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import copy
import math

import IPython
import numpy as np
import torch
import socialforce

(corridor)=
# Corridor

We want to qualitatively study the impact of non-standard Social Force potentials.

In [ ]:
def initial_state_corridor(n):
    _ = torch.manual_seed(42)

    # first n people go right, second n people go left
    state = torch.zeros((n * 2, 6))

    # positions
    state[:n, 0:2] = ((torch.rand((n, 2)) - 0.5) * 2.0) * torch.tensor([25.0, 4.5])
    state[n:, 0:2] = ((torch.rand((n, 2)) - 0.5) * 2.0) * torch.tensor([25.0, 4.5])

    # velocity
    state[:n, 2] = torch.normal(torch.full((n,), 1.34), 0.26)
    state[n:, 2] = torch.normal(torch.full((n,), -1.34), 0.26)

    # x destination
    state[:n, 4] = 100.0
    state[n:, 4] = -100.0

    return state

initial_state = initial_state_corridor(60)


The space is just two walls at $y=5.0m$ and $y=-5.0m$. To avoid boundary effects,
the walls extend beyond the periodic boundaries at $x=-25m$ and $x=25m$.

In [ ]:

upper_wall = torch.stack([torch.linspace(-30, 30, 600), torch.full((600,), 5)], -1)
lower_wall = torch.stack([torch.linspace(-30, 30, 600), torch.full((600,), -5)], -1)
ped_space = socialforce.PedSpacePotential([upper_wall, lower_wall])


## Rereference Potential

Standard SF:

In [ ]:
ped_ped = socialforce.potentials.PedPedPotential()

In [ ]:
# HIDE CODE
simulator = socialforce.Simulator(ped_ped=ped_ped, ped_space=ped_space,
                                  oversampling=2, delta_t=0.08)
simulator.integrator = socialforce.simulator.PeriodicBoundary(
    simulator.integrator, x_boundary=[-25.0, 25.0])

with torch.no_grad():
    states = simulator.run(initial_state, 250)
states_sf = states  # backup for analysis later

with socialforce.show.track_canvas(ncols=2, figsize=(12, 2), tight_layout=False) as (ax1, ax2):
    socialforce.show.states(ax1, states[0:1], monochrome=True)
    socialforce.show.space(ax1, ped_space)
    ax1.text(0.1, 0.1, '$t = 0s$', transform=ax1.transAxes)
    ax1.set_xlim(-25, 25)

    socialforce.show.states(ax2, states[249:250], monochrome=True)
    socialforce.show.space(ax2, ped_space)
    ax2.text(0.1, 0.1, '$t = 20s$', transform=ax2.transAxes)
    ax2.set_xlim(-25, 25)

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas(figsize=(6, 2), tight_layout=False, show=False, dpi=130) as ax:
    ax.set_xlim(-25, 25)
    socialforce.show.space(ax, ped_space)
    video = socialforce.show.state_animation(ax, states, delta_t=0.08, monochrome=True).to_html5_video()

IPython.display.HTML(video)


## Diamond Potential

In [ ]:
ped_ped = socialforce.potentials.PedPedPotentialDiamond(sigma=0.5)

In [ ]:
# HIDE CODE
simulator = socialforce.Simulator(ped_ped=ped_ped, ped_space=ped_space,
                                  oversampling=2, delta_t=0.08)
simulator.integrator = socialforce.simulator.PeriodicBoundary(
    simulator.integrator, x_boundary=[-25.0, 25.0])

with torch.no_grad():
    states = simulator.run(initial_state, 250)
states_diamond = states  # backup for analysis later

with socialforce.show.track_canvas(ncols=2, figsize=(12, 2), tight_layout=False) as (ax1, ax2):
    socialforce.show.states(ax1, states[0:1], monochrome=True)
    socialforce.show.space(ax1, ped_space)
    ax1.text(0.1, 0.1, '$t = 0s$', transform=ax1.transAxes)
    ax1.set_xlim(-25, 25)

    socialforce.show.states(ax2, states[249:250], monochrome=True)
    socialforce.show.space(ax2, ped_space)
    ax2.text(0.1, 0.1, '$t = 20s$', transform=ax2.transAxes)
    ax2.set_xlim(-25, 25)
    

In [ ]:
# HIDE CODE
with socialforce.show.track_canvas(figsize=(6, 2), tight_layout=False, show=False, dpi=130) as ax:
    ax.set_xlim(-25, 25)
    socialforce.show.space(ax, ped_space)
    video = socialforce.show.state_animation(ax, states, delta_t=0.08, monochrome=True).to_html5_video()

IPython.display.HTML(video)


## Analysis

In [ ]:
def relative_speeds(states):
    speeds = np.linalg.norm(states[:, :, 2:4], axis=-1)
    preferred = states[:, :, 9]
    relative = speeds / preferred

    # ignore the first 50
    relative = relative[50:]
    return relative.reshape(-1)

with socialforce.show.canvas() as ax:
    r_sf = relative_speeds(states_sf)
    r_diamond = relative_speeds(states_diamond)
    ax.hist([r_sf, r_diamond], bins=30, range=(0.8, 1.35), density=True, label='Social Force')
    ax.legend()
    ax.set_xlabel('$v / v_{preferred}$ [m/s]')